# Python `subprocess` Module Guide

## 1. Overview

The `subprocess` module lets your Python program communicate with other programs (like `git`, `ls`, or custom tools). Think of it as calling a friend to do a job and listening to what they say back.

## 2. Getting Started with `subprocess.run()`

### 2.1 The Main Function

`subprocess.run(...)` is your primary tool for running external commands:

- Commands are passed as a list: `["prog", "arg1", "arg2"]`
- Returns a `CompletedProcess` object containing:
  - `.returncode` — exit status (0 typically means success)
  - `.stdout` — captured standard output (if requested)
  - `.stderr` — captured error output (if requested)

### 2.2 Basic Example

Run `python --version` and capture the output:

In [36]:
import subprocess
import sys

cp = subprocess.run([sys.executable, "--version"], capture_output=True, text=True)

print("Return Code:", cp.returncode)  # Should be 0 for success
print("Standard Output:", cp.stdout)  # Should print "Python X.Y.Z\n"
print("Standard Error:", cp.stderr)  # Usually empty

Return Code: 0
Standard Output: Python 3.14.0

Standard Error: 


### 2.3 Why Use a List of Arguments?

Using a list is safer and avoids shell parsing surprises. Avoid `shell=True` unless absolutely necessary (e.g., Windows shell built-ins like `cd`).

In [37]:
import subprocess

# Using shell=True with a string command
cp = subprocess.run(["cd /somepath"], shell=True, capture_output=True, text=True)

print("Return Code:", cp.returncode)
print("Standard Output:", cp.stdout)
print("Standard Error:", cp.stderr)

Return Code: 1
Standard Output: 
Standard Error: The system cannot find the path specified.



## 3. Essential Parameters

### 3.1 Core Parameters Reference

| Parameter        | Default    | Description                                        |
| ---------------- | ---------- | -------------------------------------------------- |
| `args`           | (required) | Command to execute as a list: `["git", "status"]`  |
| `check`          | `False`    | Raises `CalledProcessError` on non-zero exit code  |
| `capture_output` | `False`    | Captures both `stdout` and `stderr`                |
| `text`           | `False`    | Returns output as strings instead of bytes         |
| `timeout`        | `None`     | Maximum seconds to wait (raises `TimeoutExpired`)  |
| `encoding`       | `None`     | Character encoding for text mode (e.g., `"utf-8"`) |

> **Note:** `capture_output` was added in Python 3.7. For older versions, use `stdout=subprocess.PIPE, stderr=subprocess.PIPE`.

### 3.2 Understanding text=True vs text=False

#### With text=True (String Output)

In [38]:
import subprocess
import sys

cp_as_str: subprocess.CompletedProcess[str] = subprocess.run(
    [sys.executable, "--version"], capture_output=True, text=True, encoding="utf-8"
)

print("Return Code:", cp_as_str.returncode)
print("Standard Output (strings):", cp_as_str.stdout)
print("Standard Error (strings):", cp_as_str.stderr)

Return Code: 0
Standard Output (strings): Python 3.14.0

Standard Error (strings): 


#### With text=False (Bytes Output)

In [39]:
cp_as_bytes: subprocess.CompletedProcess[bytes] = subprocess.run(
    [sys.executable, "--version"],
    capture_output=True,
    # text=False by default
)

print("Return Code:", cp_as_bytes.returncode)
print("Standard Output (bytes):", cp_as_bytes.stdout)
print("Standard Error (bytes):", cp_as_bytes.stderr)

Return Code: 0
Standard Output (bytes): b'Python 3.14.0\r\n'
Standard Error (bytes): b''


### 3.3 When to Use Each Mode

- **Use `text=True`**: When working with text output (logs, version strings, human-readable data)
- **Use `text=False`**: When handling binary data (images, archives, precise byte-level operations)
- **Tip**: Explicitly set `encoding="utf-8"` for consistent, reproducible decoding

## 4. Handling Success and Failure

### 4.1 Pattern A: Manual Return Code Inspection

Use this pattern when non-zero return codes are expected or you want to handle them manually:

In [40]:
import subprocess

cp: subprocess.CompletedProcess[str] = subprocess.run(
    ["git", "status"],
    capture_output=True,
    text=True,
    # check=False by default
)

if cp.returncode == 0:
    print("Success:", cp.stdout)
else:
    print("Failed with code", cp.returncode)
    print("Error:", cp.stderr)

Success: On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../../learn-css/.gitignore
	modified:   ../../learn-css/package.json
	modified:   ../../learn-css/src/00-introduction.html
	deleted:    ../../learn-css/src/static/global.css

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../../learn-css/src/static/.gitignore
	../../learn-css/src/static/input.css
	./

no changes added to commit (use "git add" and/or "git commit -a")



### 4.2 Pattern B: Exception-Based Error Handling

Use this pattern when non-zero exits should be treated as exceptions (fail-fast approach):

In [41]:
import subprocess

try:
    cp: subprocess.CompletedProcess[str] = subprocess.run(
        ["git", "status"], capture_output=True, text=True, check=True
    )
except subprocess.CalledProcessError as e:
    print("Command failed:", e.cmd)
    print("Return Code:", e.returncode)
    print("Standard Output:", e.stdout)
    print("Standard Error:", e.stderr)
else:
    print("Command succeeded. Output:")
    print(cp.stdout)

Command succeeded. Output:
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../../learn-css/.gitignore
	modified:   ../../learn-css/package.json
	modified:   ../../learn-css/src/00-introduction.html
	deleted:    ../../learn-css/src/static/global.css

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../../learn-css/src/static/.gitignore
	../../learn-css/src/static/input.css
	./

no changes added to commit (use "git add" and/or "git commit -a")



### 4.3 Choosing the Right Pattern

| Use Pattern A When                  | Use Pattern B When                   |
| ----------------------------------- | ------------------------------------ |
| Non-zero exits are normal           | Failures should stop execution       |
| You need custom logic per exit code | You want automatic error propagation |
| You're checking multiple conditions | You want clean success path code     |

**Key Difference:**

- Pattern A: `cp` is always a `CompletedProcess` object
- Pattern B: `e` only exists when `check=True` and the command fails

## 5. Exception Handling

### 5.1 Common subprocess Exceptions

| Exception                       | When It Occurs                                   | Parent Class      |
| ------------------------------- | ------------------------------------------------ | ----------------- |
| `FileNotFoundError`             | Executable not found (e.g., `git` not installed) | `OSError`         |
| `PermissionError`               | File exists but lacks execute permission         | `OSError`         |
| `subprocess.TimeoutExpired`     | Process exceeds `timeout` seconds                | `SubprocessError` |
| `subprocess.CalledProcessError` | Non-zero exit when `check=True`                  | `SubprocessError` |
| `OSError`                       | General OS-level errors                          | Base exception    |
| `subprocess.SubprocessError`    | Base class for subprocess-specific errors        | Base exception    |

### 5.2 Comprehensive Exception Handling Example

In [42]:
import subprocess

try:
    cp: subprocess.CompletedProcess[str] = subprocess.run(
        ["myprogram"], check=True, capture_output=True, text=True, timeout=5
    )
except FileNotFoundError as fe:
    print(f"Program not found: {fe.filename}")
    print("Is it installed and in your PATH?")

except PermissionError:
    print("Permission denied: cannot execute 'myprogram'")

except subprocess.TimeoutExpired as te:
    print(f"Timed out after {te.timeout} seconds")
    print(f"Partial output: {te.stdout}")

except subprocess.CalledProcessError as cpe:
    print(f"Command {cpe.cmd} failed")
    print(f"Return code: {cpe.returncode}")
    print(f"Error output: {cpe.stderr}")

else:
    print("Success! Output:")
    print(cp.stdout)

Program not found: None
Is it installed and in your PATH?


> **Tip:** Order exception handlers from specific to general (`FileNotFoundError` before `OSError`) to catch the most precise error type first.

## 6. Best Practices

### 6.1 Security and Safety

- ✅ **Always use a list of arguments**: `["prog", "arg1", "arg2"]`
- ❌ **Avoid `shell=True`**: Prevents shell injection vulnerabilities
- ✅ **Validate user input**: Never pass unsanitized user input to commands

### 6.2 Output Handling

- Use `capture_output=True` to capture both stdout and stderr
- Set `text=True` for text processing, `text=False` for binary data
- Specify `encoding="utf-8"` for reproducible text decoding

### 6.3 Error Management

- Use `check=True` when failures should stop your script
- Catch `CalledProcessError` to log detailed failure information
- Use manual `returncode` checking when non-zero exits are expected

### 6.4 Timeout Handling

- Always set `timeout=...` for potentially long-running processes
- Catch `TimeoutExpired` to access partial output via `.stdout` and `.stderr`

### 6.5 Exception Handling

- Catch specific exceptions rather than broad `Exception`
- Order handlers from specific to general
- Always log or handle errors appropriately

### 6.6 Compatibility

- `capture_output=True` requires Python 3.7+
- For older versions: `stdout=subprocess.PIPE, stderr=subprocess.PIPE`

## 7. Quick Reference

```python
import subprocess

# Simple command
subprocess.run(["git", "-v"])

# Capture output as text
cp = subprocess.run(["git", "status"], capture_output=True, text=True, encoding="utf-8")

# Fail-fast on errors
try:
    cp: subprocess.CompletedProcess[str] = subprocess.run(
        ["myprogram"],
        check=True,
        capture_output=True,
        text=True,
        encoding="utf-8",
        timeout=300,
    )
except subprocess.CalledProcessError as e:
    print(e.cmd)
    print(e.returncode)
    print(e.stderr)
else:
    print(cp.stdout)

# Handle errors manually
cp = subprocess.run(["make", "build"], capture_output=True, text=True)
if cp.returncode != 0:
    print(f"Build failed: {cp.stderr}")
```

## 8. Further Reading

- **Official Documentation:** [subprocess — Python Docs](https://docs.python.org/3/library/subprocess.html)
- **Related Topics:** `os.system()`, `shlex`, `pathlib`